# 7.4. 含并行连结的网络（GoogLeNet）
- **目录**
  - 7.4.1 Inception块
  - 7.4.2 GoogLeNet模型
  - 7.4.3 训练模型


- **GoogLeNet**2014年出现，吸收了NiN中串联网络的思想，并在此基础上做了改进。
- 该模型的重点是解决了什么样大小的卷积核最合适的问题。
- 以前流行的网络使用小到$1 \times 1$，大到$11 \times 11$的卷积核。
- 本文的一个观点是，有时**使用不同大小的卷积核组合是有利的**。
- 在本节介绍一个稍微简化的GoogLeNet版本。

------------
- **说明：**
- **（1）何为“NiN中串联网络的思想”？**
  - 在传统的卷积神经网络（CNN）中，卷积核的大小通常为 $3 \times 3$ 或 $5 \times 5$ 等。
    - NiN 模型引入了使用 $1 \times 1$ 的卷积核，这种非常小的卷积核在每个位置上只与输入的深度方向（通道数）进行点乘运算。
    - 这实际上是对每个像素点的所有通道进行线性组合，从而实现了跨通道的信息融合。
  - 在 NiN 模型中，卷积层后面紧跟着一个 $1 \times 1$ 的卷积层，这种结构被称为“串联”网络。
    - 这个操作可以理解为在每一层之后进行一个小的全连接操作，但仍然保持空间信息（因为它是逐像素进行的）。
    - 这种方法有效地增加了网络的表达能力，并且减少了参数量。
  - GoogLeNet 吸收了 NiN 的这种思想，并将其扩展到 Inception 模块中。
    - 在 Inception 模块中，使用了多个不同大小的卷积核（例如 $1 \times 1$、$3 \times 3$ 和 $5 \times 5$），并将它们的输出拼接在一起。
    - 这种方法允许模型在同一层次上同时捕捉到不同尺度的特征，同时通过 $1 \times 1$ 卷积来减少计算量和参数量。
  - 串联网络的计算过程
- **（2） $1 \times 1$ 卷积层实现全连接运算示例**
  - 假设输入特征图的形状为 $(2, 3, 3)$（即通道数为2，高为3，宽为3），卷积核的形状为 $(1, 2, 1, 1)$（即单输出通道），具体数值如下：
    - **输入特征图 $\mathbf{X}$**：
    $$
\mathbf{X} =
\begin{bmatrix}
\begin{bmatrix}
1 & 3 & 5 \\
7 & 9 & 11 \\
13 & 15 & 17
\end{bmatrix},
\begin{bmatrix}
2 & 4 & 6 \\
8 & 10 & 12 \\
14 & 16 & 18
\end{bmatrix}
\end{bmatrix}
    $$

    - **1x1卷积核的权重 $\mathbf{w}$**：

    $$
\mathbf{w} = \begin{bmatrix} 0.5 & -0.5 \end{bmatrix}
    $$
   - 对于位置 $(h=1, w=1)$ 处的像素点，计算过程如下：
     - 1. **线性组合计算 $ y_{1,1} $**：
    $$
y_{1,1} = 0.5 \cdot 1 + (-0.5) \cdot 2 = 0.5 - 1 = -0.5
    $$
     - 2. **ReLU激活**：
    $$
z_{1,1} = \max(0, -0.5) = 0
    $$

-------------

## 7.4.1 Inception块

- 在GoogLeNet中，基本的卷积块被称为**Inception块（Inception block）**。
  - 这很可能得名于电影《盗梦空间》（Inception），因为电影中的一句话“我们需要走得更深”（“We need to go deeper”）。

<center>
    <img src="../img/inception.svg" alt="Inception块的架构">
</center>
<center>
    图7.4.1 Inception块的架构
</center>

- 如图7.4.1所示，Inception块由**四条并行路径组成**。
  - 前三条路径使用窗口大小为$1\times 1$、$3\times 3$和$5\times 5$的卷积层，从不同空间大小中提取信息。
  - 中间的两条路径在输入上执行$1\times 1$卷积，以减少通道数，从而降低模型的复杂性。
  - 第四条路径使用$3\times 3$最大池化层，然后使用$1\times 1$卷积层来改变通道数。
  - 这四条路径都使用合适的填充来使输入与输出的高和宽一致，最后将每条线路的输出在通道维度上连结，并构成Inception块的输出。
  - 在Inception块中，通常**调整的超参数是每层输出通道数**。

In [20]:
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

'''
注意Conv2d的输出通道数决定了卷积核的多少，也是Conv2d输出的通道数。
'''
class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    # Inception(192, 64, (96, 128), (16, 32), 32)为例
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大池化层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        ## 此处很重要，必须保证4个路径输出的第3和4维形状相同
        return torch.cat((p1, p2, p3, p4), dim=1)

- 那么为什么GoogLeNet这个网络如此有效呢？
  - 首先考虑一下滤波器（filter）的组合，它们可以用各种滤波器尺寸探索图像，这意味着**不同大小的滤波器可以有效地识别不同范围的图像细节**。
  - 同时，可以为不同的滤波器分配不同数量的参数。

-----------
- **说明：Inception输出形状以及通道维的拼接，以Inception(192, 64, (96, 128), (16, 32), 32)为例**
  - **假设输入张量 `x` 的形状为 `(batch_size, 192, H, W)`**
    - 其中 `batch_size` 是批量大小，
    - `192` 是输入通道数，
    - `H` 和 `W` 是输入的高和宽。
  - **路径 1 (`p1`)**
    - 子层 `p1_1`:
      - 输入形状: `(batch_size, 192, H, W)`
      - 卷积核大小: `1x1`
      - 输出通道数: `64`
      - 输出形状: `(batch_size, 64, H, W)`
  - **路径 2 (`p2`)**
    - 子层 `p2_1`:
      - 输入形状: `(batch_size, 192, H, W)`
      - 卷积核大小: `1x1`
      - 输出通道数: `96`
      - 输出形状: `(batch_size, 96, H, W)`
    - 子层 `p2_2`:
      - 输入形状: `(batch_size, 96, H, W)`
      - 卷积核大小: `3x3`
      - Padding: `1`（保持输入和输出的空间尺寸不变）
      - 输出通道数: `128`
      - 输出形状: `(batch_size, 128, H, W)`
  - **路径 3 (`p3`)**
    - 子层 `p3_1`:
      - 输入形状: `(batch_size, 192, H, W)`
      - 卷积核大小: `1x1`
      - 输出通道数: `16`
      - 输出形状: `(batch_size, 16, H, W)`
    - 子层 `p3_2`:
      - 输入形状: `(batch_size, 16, H, W)`
      - 卷积核大小: `5x5`
      - Padding: `2`（保持输入和输出的空间尺寸不变）
      - 输出通道数: `32`
      - 输出形状: `(batch_size, 32, H, W)`
  - **路径 4 (`p4`)**
    - 子层 `p4_1`:
      - 输入形状: `(batch_size, 192, H, W)`
      - 最大池化核大小: `3x3`
      - Padding: `1`（保持输入和输出的空间尺寸不变）
      - 最大池化层输出形状: `(batch_size, 192, H, W)`
    - 子层 `p4_2`:
      - 输入形状: `(batch_size, 192, H, W)`
      - 卷积核大小: `1x1`
      - 输出通道数: `32`
      - 输出形状: `(batch_size, 32, H, W)`
  - **汇总**
    - `p1` 的输出形状: `(batch_size, 64, H, W)`
    - `p2` 的输出形状: `(batch_size, 128, H, W)`
    - `p3` 的输出形状: `(batch_size, 32, H, W)`
    - `p4` 的输出形状: `(batch_size, 32, H, W)`
  - 最终，Inception块的输出是将这些路径的输出在通道维度上进行连接:
    - **输出形状**: `(batch_size, 64 + 128 + 32 + 32, H, W) = (batch_size, 256, H, W)`
----------------------

## 7.4.2 GoogLeNet模型

- 如图7.4.2所示，GoogLeNet一共使用9个Inception块和全局平均池化层的堆叠来生成其估计值。
- Inception块之间的最大池化层可降低维度。
- 第一个模块类似于AlexNet和LeNet，Inception块的组合从VGG继承，全局平均池化层避免了在最后使用全连接层。

<center>
    <img src="../img/inception-full.svg" alt="GoogLeNet架构">
</center>
<center>
    图7.4.2 GoogLeNet架构
</center>

- 现在逐一实现GoogLeNet的每个模块。
- 第一个模块使用64个通道、$7\times 7$卷积层。

In [21]:
## 第1,2格：7*7卷积层与3*3最大池化层
## 卷积层将特征图大小减半，然后最大层化层再减半，通过b1后特征图的大小减小为输入的1/4
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),## 第1格
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1)) ## 第2格

In [22]:
## 根据6.6节的公式计算结果与下述结果一致
## 后面各层类似
X = torch.rand(size=(1, 1, 96, 96))
for layer in  nn.Sequential(b1):
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 24, 24])


- 第二个模块使用两个卷积层：
  - 第一个卷积层是64个通道、$1\times 1$卷积层；
  - 第二个卷积层使用将通道数量增加三倍的$3\times 3$卷积层。
  - 对应于Inception块中的第二条路径。


In [23]:
## 分别实现第3,4,5格
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),## 第3格，输出大小不变：24*24
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1), ## 第4格，输出大小不变：24*24
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1)) ## 第5格，输出大小变为：12*12

- 第三个模块串联两个完整的Inception块。
  - 第一个Inception块的输出通道数为$64+128+32+32=256$，四个路径之间的输出通道数量比为$64:128:32:32=2:4:1:1$。
  - 第二个和第三个路径首先将输入通道的数量分别减少到$96/192=1/2$和$16/192=1/12$，然后连接第二个卷积层。
  - 第二个Inception块的输出通道数增加到$128+192+96+64=480$，四个路径之间的输出通道数量比为$128:192:96:64 = 4:6:3:2$。
  - 第二条和第三条路径首先将输入通道的数量分别减少到$128/256=1/2$和$32/256=1/8$。


In [25]:
## 第6,7格，第6格式Inception块
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32), ## 第6格第1个Inception
                   Inception(256, 128, (128, 192), (32, 96), 64), ## 第6格第2个Inception
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1)) ## 第7格

- 第四模块更加复杂，它串联了5个Inception块，其输出通道数分别是$192+208+48+64=512$、$160+224+64+64=512$、$128+256+64+64=512$、$112+288+64+64=528$和$256+320+128+128=832$。
  - 首先是含$3×3$卷积层的第二条路径输出最多通道，
  - 其次是仅含$1×1$卷积层的第一条路径，
  - 之后是含$5×5$卷积层的第三条路径和含$3×3$最大池化层的第四条路径。
  - 其中第二、第三条路径都会先按比例减小通道数。
  - 这些比例在各个Inception块中都略有不同。


In [26]:
## 第8,9格，其中第8格5个Inception构成的块
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64), ## 第8格第1个Inception
                   Inception(512, 160, (112, 224), (24, 64), 64), ## 第8格第2个Inception
                   Inception(512, 128, (128, 256), (24, 64), 64), ## 第8格第3个Inception
                   Inception(512, 112, (144, 288), (32, 64), 64), ## 第8格第4个Inception
                   Inception(528, 256, (160, 320), (32, 128), 128), ## 第8格第5个Inception
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1)) ## 第9格

- 第五模块包含输出通道数为$256+320+128+128=832$和$384+384+128+128=1024$的两个Inception块。
  - 其中每条路径通道数的分配思路和第三、第四模块中的一致，只是在具体数值上有所不同。
  - 需要注意的是，第五模块的后面紧跟输出层，该模块同NiN一样使用全局平均池化层，将每个通道的高和宽变成1。
  - 最后我们将输出变成二维数组，再接上一个输出个数为标签类别数的全连接层。

In [27]:
## 第10,11格，其中第10格是两个Inception构成的块
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128), ## 第10格第1个Inception
                   Inception(832, 384, (192, 384), (48, 128), 128), ## 第10格第2个Inception
                   nn.AdaptiveAvgPool2d((1,1)), ## 第11格
                   nn.Flatten())

net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10)) ## 最后第12格是一个全连接层

- GoogLeNet模型的计算复杂，而且不如VGG那样便于修改通道数。
- 为了使Fashion-MNIST上的训练短小精悍，下例将输入的高和宽从224降到96，这简化了计算。
- 下面演示各个模块输出的形状变化。


In [28]:
X = torch.rand(size=(1, 1, 96, 96))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])


## 7.4.3 训练模型

- 使用Fashion-MNIST数据集来训练模型。
- 在训练之前，将图片转换为$96 \times 96$分辨率。

In [2]:
import time
import numpy as np
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
start = time.time()
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())
end = time.time()
print(f'3090ti上模型训练耗时：{int(np.floor((end-start)/60))}分钟{(int(end-start)%60) }秒')

<img src='..\img\7_4_1.png' height=350 width=350>

## 小结

* Inception块相当于一个有4条路径的子网络。它通过不同窗口形状的卷积层和最大池化层来并行抽取信息，并使用$1×1$卷积层减少每像素级别上的通道维数从而降低模型复杂度。
*  GoogLeNet将多个设计精细的Inception块与其他层（卷积层、全连接层）串联起来。其中Inception块的通道数分配之比是在ImageNet数据集上通过大量的实验得来的。
* GoogLeNet和它的后继者们一度是ImageNet上最有效的模型之一：它以较低的计算复杂度提供了类似的测试精度。